逆累积分布法生成随机数

In [1]:
import java.util.concurrent.ThreadLocalRandom //另一种随机数是breeze的distributions包

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1671519252453)
SparkSession available as 'spark'


import java.util.concurrent.ThreadLocalRandom


In [3]:
val r=ThreadLocalRandom.current

r: java.util.concurrent.ThreadLocalRandom = java.util.concurrent.ThreadLocalRandom@7d703d33


In [3]:
r.nextDouble//U(0,1)

res0: Double = 0.954923907667987


In [4]:
r.nextGaussian//N(0,1)

res1: Double = -0.6759082483858904


In [5]:
val ExpRV=(1 to 1000).map(x=>r.nextDouble)
val out=ExpRV.map({-math.log(_)})//用x写报错

ExpRV: scala.collection.immutable.IndexedSeq[Double] = Vector(0.8083377686022807, 0.020750980584776202, 0.5323742337674121, 0.14370976686822423, 0.8173266546850708, 0.895840203058051, 0.07393837285188787, 0.774132858462939, 0.10972244266098385, 0.7323872125796077, 0.2428318595123039, 0.4340209380086303, 0.7152488206687321, 0.6012440347741536, 0.7810324465549444, 0.12841307777323308, 0.8749842929188758, 0.606503281021103, 0.0661782028539728, 0.07066445976383151, 0.4583141772402657, 0.32435001864197344, 0.5542152435228958, 0.37581695057603315, 0.6669126440607918, 0.14935482974401582, 0.3218854179891407, 0.22355079760636454, 0.839942159532019, 0.4404344629280511, 0.8402419974105716, 0.8829426189498852, 0.7688658718640758, 0.3825509710491982, 0.7814908276282774, 0.36970913107431136, 0.68576...


In [6]:
(1 to 1000).map(x=>r.nextDouble).map(x=> -math.log(x))//这样也可以，但是=>与-之间必须有空格（=>两端都加空格最好）

res2: scala.collection.immutable.IndexedSeq[Double] = Vector(0.09734327131164676, 1.1528342757837546, 0.4762408016262779, 2.4801478573691345, 1.0798227536542053, 0.12554108446963186, 0.7458252603383904, 0.045347436619583355, 0.35142681550437505, 0.10859488032450453, 0.3628363204793406, 1.373550002351918, 0.4756181865632218, 2.1984912651471715, 0.02688844904988542, 2.2275528878741477, 0.6118081964661439, 1.2492519876224784, 1.032805653747642, 0.8208275524512111, 0.8068691464528482, 0.7484908136064624, 0.10849852690384588, 0.5437884729646159, 0.20120828190027445, 0.640661179254952, 1.6879078599760613, 0.08468566755953424, 0.7019215849071299, 0.030111248379577394, 1.3318160264446515, 3.043340946370601, 0.015885087342444034, 7.286909061962676, 0.4176327795670509, 0.2453316031059628, 3.37918...


In [4]:
(1 to 1000).map(x=>r.nextDouble).map(x=> -math.log(x)).toArray[Double]

res1: Array[Double] = Array(1.2543553059051624, 0.18210512288365377, 0.6376490517355058, 1.125761218702395, 3.4224253570255936, 0.32695526795721463, 4.64800956307554, 0.3433439480034596, 1.6108599466413704, 2.1178527371809572, 0.023127386183624594, 1.3414944133011293, 2.228220062656898, 0.3130585161607245, 0.6163237789024494, 1.9307981987963907, 0.418156767011687, 0.5656339385979047, 0.8607135297119252, 1.2355712357740536, 1.3249277974437328, 0.04254011502398247, 0.5124243010694624, 0.23730172995888915, 1.1310151889592448, 5.715326631370734, 0.052681249198973795, 0.8862570452558585, 2.1970028124153513, 4.199408693319819, 0.27012281306096575, 0.6319224283216837, 1.0736892919735528, 0.5543996083507651, 0.09019815152246825, 0.9083221373610163, 0.4644141387069685, 0.013211776044143099, 0.70...


In [7]:
ExpRV.min
ExpRV.reverse
ExpRV.sorted
ExpRV.sum

res3: Double = 500.95089830732036


In [8]:
Array(1,2,3).union(Array(4,5,6))//合并数组

res4: Array[Int] = Array(1, 2, 3, 4, 5, 6)


案例：从回归模型中模拟数据

In [9]:
import java.util.concurrent.ThreadLocalRandom

import java.util.concurrent.ThreadLocalRandom


In [10]:
val RowSize=sc.broadcast(200)//将这个值200广播到各个节点上去，这样每个节点都可以读取这个值

RowSize: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(0)


In [11]:
RowSize.value//收取广播出去的值

res5: Int = 200


In [12]:
val ColumnSize=sc.broadcast(5)
val RowLength=sc.broadcast(2)//将要生成的400个数据分成两拨，同时用两个节点生成
val ColumnLength=sc.broadcast(1000)//列向量分块方法：5个一组，共1000块
val NonZeroLength=sc.broadcast(10)//非零变量个数
val p=ColumnSize.value*ColumnLength.value

ColumnSize: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(1)
RowLength: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(2)
ColumnLength: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(3)
NonZeroLength: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(4)
p: Int = 5000


In [13]:
val betta=(1 to p).map(_.toDouble).toArray[Double].map(i => { if(i <= NonZeroLength.value ) 2.0 else 0.0 })//前10个数为2，其他数都是0
val MyBeta=sc.broadcast(betta)

betta: Array[Double] = Array(2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...


In [14]:
var sigma=1.0
val Sigma=sc.broadcast(sigma)

sigma: Double = 1.0
Sigma: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(6)


In [15]:
var indices=0 until RowLength.value

indices: scala.collection.immutable.Range = Range 0 until 2


In [16]:
var ParallelIndices=sc.parallelize(indices,indices.length)
//投射到RDD里面去，后一个参数不加也可

ParallelIndices: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:28


In [17]:
ParallelIndices.collect()

22/10/25 18:43:47 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


res6: Array[Int] = Array(0, 1)


In [18]:
def rn(n:Int)=(0 until n).map(x => r.nextGaussian).toArray[Double]

rn: (n: Int)Array[Double]


In [19]:
rn(5)//容量为5的标准正态样本

res7: Array[Double] = Array(-0.7983481678566424, 1.595845292169618, -0.06576379284920383, -2.494737082702359, 0.49965859790756306)


In [20]:
ParallelIndices.map(s=>"ssf").collect()

res8: Array[String] = Array(ssf, ssf)


In [21]:
ParallelIndices.collect()

res9: Array[Int] = Array(0, 1)


In [56]:
//生成线性模型
val lines=ParallelIndices.map(s => {  //lines两个元素，每个元素化身一个巨长的字符串，分别是两批数据
    val r=ThreadLocalRandom.current  //生成随机数
    val beta=MyBeta.value
    val sigma=Sigma.value
    val rowsize=RowSize.value
    val columnsize=ColumnSize.value
    val columnlength=ColumnLength.value
    var lines=new Array[String](rowsize) //声明方法，中括号写类型，圆括号写长度
     
    val p=columnsize*columnlength
    for(i <- 0 until rowsize){ //rowsize=200
        var line=""; //空字符串 
        var y=0.0;
        //每一行五列，每列1000块，在每一行上进行循环（一小块是5个元）
        for(j <- 0 until columnlength){  //columnlength=1000
            var x=rn(columnsize) //5个正态随机数，作为第i行第j块的那5个
            for(k <- 0 until columnsize){ //columnsize=5
                y+= beta(j*columnsize+k)*x(k) //将这一块上的beta加到y上去
            }
            var segment=x.map("%.4f" format _).reduce(_+" "+_) //特别注意format前后的两个空格，表示保留四位小数，还是一个字符串
            line=line+","+segment //将每一块用逗号隔开
        }//完成了每一块的操作，每一块的数都加到了y上去，现在是yi=beta.t*xi
        y += sigma*r.nextGaussian //完全生成了出来了，这是加上了误差项
        lines(i)="%.4f".format(y)+line+"\n" //生成文件的话一定要加上换行符，一个yi一个换行符（一个yi带着它的5000个xi的分量占一行）
    }
    (s,lines.reduce(_+_)) //此为返回值，把这两百个元素合并起来；元组的第一个数为标签。元组的元素可为不同的类型
}
)
//这是200*2个，在两个节点上同时跑，每个节点分别输出200个

lines: org.apache.spark.rdd.RDD[(Int, String)] = MapPartitionsRDD[7] at map at <console>:44


In [58]:
lines.collect()(0)

res30: (Int, String) =
(0,"1.6789,1.7059 -0.2902 -0.3960 0.5969 0.7093,-0.2458 -0.0008 -0.2863 0.1393 -0.9587,-0.4540 0.0882 0.6548 0.4091 -0.5121,1.9541 0.4959 0.2188 -0.9891 -0.3671,2.2453 0.5475 0.4803 0.9900 -1.8211,0.3663 -0.5530 1.1532 1.4787 0.3088,0.2294 -0.1109 -0.4427 0.3598 1.1512,0.2577 -0.4359 1.0894 0.4249 0.6350,1.6048 0.1075 1.4508 0.4237 0.0486,-0.9324 0.1356 0.3881 0.4130 -0.5132,-0.4184 -0.1880 1.3410 0.7902 0.7569,-0.5215 0.9675 -1.2908 -0.4222 -0.1477,-1.4972 -0.1114 0.0711 -0.2748 0.3650,-1.5445 -0.7514 1.5753 -0.3137 0.7967,-0.1889 0.8584 0.3029 -0.8941 -1.4448,0.9056 1.6234 -0.0075 0.9042 -1.3147,0.2367 0.8476 0.5164 2.1557 0.8417,-1.1620 -1.0969 -1.8698 -0.6054 2.9503,-0.8032 0.1674 0.0216 0.4175 1.2890,-0.4419 -0.5687 -0.6088 -0.3636 0.3837,0.6314 0.8590 1.935...


In [39]:
import java.io._

import java.io._


In [42]:
val pw=new PrintWriter(new File("aa.txt"))
//改了之后不用这种输出方式了，用下面的分布式输出

pw: java.io.PrintWriter = java.io.PrintWriter@cc7dedd


In [44]:
pw.write(lines.collect()(0))
pw.close 

<console>: 34: error: (Int, String) does not take parameters

In [34]:
val test=lines.collect()
//每次执行这个action操作之后，都会将map的结果重新运行
//一遍，这是因为mapredece的惰性调用

test: Array[String] =
Array("-3.0554,0.3992 -0.8226 0.9089 0.5088 -0.8896,-0.7489 0.8627 -1.6104 0.1723 -1.0189,-0.7344 -1.0664 0.5134 -0.7631 0.0410,-0.0692 -0.2482 -0.2285 0.4651 0.0274,1.3147 0.3327 0.2312 1.1329 -0.6835,-2.6022 0.8545 1.1494 -1.2150 0.6491,0.8527 -0.7611 1.5499 -0.6111 0.0430,0.4972 1.5369 0.6141 -0.5157 -0.1350,0.3243 1.5084 -1.1571 1.1455 1.9968,-0.1812 -1.2569 -0.0022 -0.1753 0.1230,0.3780 -0.5238 -1.8470 0.5336 -0.3593,0.0326 -0.2302 -0.7535 0.5338 -1.0093,0.3252 1.8131 0.5433 0.5093 0.4318,1.1027 1.9018 -0.7872 1.4875 2.7303,0.3005 1.0091 -1.7802 0.4623 -0.1194,0.9419 1.5599 0.1096 -0.9917 0.7865,0.4139 -0.0072 1.2018 -0.5757 0.0151,-0.9957 -0.6968 1.2143 -0.2183 1.9217,-2.2296 2.4559 -0.0790 0.7965 -0.5737,-0.2832 -0.7451 2.0605 -0.3203 0.8023,-0.1646 -1.4922...


In [35]:
test(0)

res17: String =
"-3.0554,0.3992 -0.8226 0.9089 0.5088 -0.8896,-0.7489 0.8627 -1.6104 0.1723 -1.0189,-0.7344 -1.0664 0.5134 -0.7631 0.0410,-0.0692 -0.2482 -0.2285 0.4651 0.0274,1.3147 0.3327 0.2312 1.1329 -0.6835,-2.6022 0.8545 1.1494 -1.2150 0.6491,0.8527 -0.7611 1.5499 -0.6111 0.0430,0.4972 1.5369 0.6141 -0.5157 -0.1350,0.3243 1.5084 -1.1571 1.1455 1.9968,-0.1812 -1.2569 -0.0022 -0.1753 0.1230,0.3780 -0.5238 -1.8470 0.5336 -0.3593,0.0326 -0.2302 -0.7535 0.5338 -1.0093,0.3252 1.8131 0.5433 0.5093 0.4318,1.1027 1.9018 -0.7872 1.4875 2.7303,0.3005 1.0091 -1.7802 0.4623 -0.1194,0.9419 1.5599 0.1096 -0.9917 0.7865,0.4139 -0.0072 1.2018 -0.5757 0.0151,-0.9957 -0.6968 1.2143 -0.2183 1.9217,-2.2296 2.4559 -0.0790 0.7965 -0.5737,-0.2832 -0.7451 2.0605 -0.3203 0.8023,-0.1646 -1.4922 0.2073 -0.8...


In [36]:
test(1) //分别是两批数据

res18: String =
"-4.5216,-1.3668 -0.8251 0.7265 0.9007 -1.6569,1.1882 -0.1402 -0.0447 -0.9111 0.9437,0.3957 -0.0492 1.0718 -1.3940 0.0486,-0.3540 -0.3848 0.6255 0.3350 0.0971,-1.9029 -0.4421 -0.1827 -1.6602 0.7731,0.0683 1.5987 -1.9170 -1.1145 0.8604,-2.4075 -0.4936 -0.5690 -0.7121 -1.4901,-0.2537 0.3124 -0.9044 -0.9268 0.1172,-2.1924 0.3985 0.0057 -0.0645 -1.0286,-0.0341 0.0171 0.0449 0.6450 0.0869,-1.2219 0.8286 -0.5010 0.0849 0.9216,-2.7211 0.8486 0.6799 1.6234 0.9010,-1.1202 0.1946 -0.2099 -0.8999 -1.2004,-0.2341 0.2512 0.5085 0.0393 -0.9786,-0.5420 1.3787 0.0919 -0.9259 0.3752,-2.1662 1.4779 -0.5579 0.3869 -0.4747,0.6751 0.3024 -1.8754 -0.2213 -0.2720,-1.0769 1.2890 0.1513 1.5137 -0.8659,0.3485 0.1464 -1.9634 1.1579 1.4089,0.3658 -1.0679 0.9161 0.1286 1.3501,0.0281 -0.1589 1.1548 ...


# 分布式输出

rdd上的map和reduce是无序的
而串型结构上的是有序的（如下）

In [68]:
val test=Array("1","2","3")
test.reduce(_+" "+_)   //有序的

test: Array[String] = Array(1, 2, 3)
res35: String = 1 2 3


In [67]:
val rdd=sc.parallelize(test)
rdd.reduce(_+" "+_)  //可能无序了

rdd: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[14] at parallelize at <console>:38
res34: String = 2 1 3


In [74]:
rdd.reduce(_+" "+_)

res41: String = 3 1 2


In [60]:
val output=lines.collect().map(s => {  //这得用collect回到串型的环境中才能这样写，RDD环境中可能不支持下面这些操作
    import java.io._
    val path="LinearModel_"+s._1+".txt"; //元组的第一个元素
    val pw=new PrintWriter(new File(path));
    pw.write(s._2);
    pw.close;
    path
})

output: Array[String] = Array(LinearModel_0.txt, LinearModel_1.txt)


读取


In [61]:
val lines=sc.textFile("LinearModel_0.txt") //自动以行为单位划分为数组（划分依据：\n）

lines: org.apache.spark.rdd.RDD[String] = LinearModel_0.txt MapPartitionsRDD[9] at textFile at <console>:33


In [76]:
lines.collect()(199)//某一行

res43: String = 3.0333,1.1555 -0.1196 -1.3351 -0.7877 -0.3780,0.2028 0.1922 -1.0302 1.8730 1.2523,-0.5117 -0.2880 1.0227 -0.8421 -1.0408,-0.2378 0.5253 -1.3542 0.5088 -0.2155,-3.2974 -1.1117 0.0260 -0.5207 -1.3870,0.9820 -0.1605 0.1181 -0.1334 0.3700,1.0782 -1.7334 -0.7586 1.0780 1.6215,1.4859 0.9438 -0.3340 -1.3422 -0.4378,-1.2718 0.6993 1.2316 -0.8454 -1.1780,0.9275 -0.8754 0.6013 -1.0032 -0.1206,-1.9915 -0.5789 0.8170 -0.5879 0.3012,0.5918 -2.1678 -0.1029 -0.7201 -1.5166,-0.8512 1.2929 -0.5555 -1.2720 -0.5955,-0.6958 0.5634 -0.2329 0.1851 0.3321,0.6093 -0.9275 2.8072 -0.3385 1.4622,-0.2825 0.5858 1.2356 -1.2432 -0.5806,0.0461 -0.5518 -0.3998 0.0827 0.0114,-0.6438 -0.2523 1.0081 0.1475 -1.3749,1.5644 1.2879 3.0912 -0.0404 1.5455,-0.9975 -0.4914 0.2164 1.0340 -0.4263,0.3014 -0.3388 0.4...


In [80]:
lines.collect()(0).split(",") //生成一个Array

res47: Array[String] = Array(3.0186, -1.2721 0.3226 1.4547 -0.6254 0.1473, 0.7119 -0.9113 0.6411 0.2659 1.0420, -2.1145 -0.3983 0.5419 -0.0772 -0.8859, -0.4116 1.0925 -1.5363 0.5546 1.7638, 0.4518 1.6495 -0.9976 -0.0182 1.1329, -0.1250 2.3728 0.2329 0.1590 0.9561, 0.1744 0.9715 -0.2927 -1.0629 1.4513, -1.1215 -0.7353 1.0172 0.6979 0.8240, -1.7683 0.2096 0.7109 -0.6189 0.6227, 1.4214 0.2154 0.0395 0.5862 -0.8251, -0.9125 0.0860 0.6893 0.0551 -1.2380, -0.0299 0.9467 -0.1940 0.7572 0.5414, -0.0388 -0.4829 -0.1502 -0.6641 -0.6174, -0.7016 0.5676 0.7943 -0.8770 -1.9943, 0.6359 1.0555 0.6047 0.8436 2.2208, 0.0236 0.4454 -0.2391 0.1376 -1.6154, -0.4678 -2.0165 0.6370 -0.2124 0.9803, 1.0565 0.1127 0.2131 -0.3955 -0.3299, 1.1327 0.9496 0.6333 1.1204 0.4342, -0.2545 0.2943 -2.2979 0.2329 -1.5774,...


In [81]:
lines.collect()(0).split(",").size  //1000个x，1一个y

res48: Int = 1001


In [62]:
lines.collect().size //一个文件两百行

res31: Int = 200


下面：让X和Y分家

先用单个做个试验，后面再写map的形式

In [82]:
val Y=lines.collect()(0).split(",")(0).toDouble  //这是一个y

res49: Double = 3.0186


In [83]:
val X=lines.collect()(0).split(",").drop(1)  //删掉Array中的前一个元素，返回剩下的元素

X: Array[String] = Array(-1.2721 0.3226 1.4547 -0.6254 0.1473, 0.7119 -0.9113 0.6411 0.2659 1.0420, -2.1145 -0.3983 0.5419 -0.0772 -0.8859, -0.4116 1.0925 -1.5363 0.5546 1.7638, 0.4518 1.6495 -0.9976 -0.0182 1.1329, -0.1250 2.3728 0.2329 0.1590 0.9561, 0.1744 0.9715 -0.2927 -1.0629 1.4513, -1.1215 -0.7353 1.0172 0.6979 0.8240, -1.7683 0.2096 0.7109 -0.6189 0.6227, 1.4214 0.2154 0.0395 0.5862 -0.8251, -0.9125 0.0860 0.6893 0.0551 -1.2380, -0.0299 0.9467 -0.1940 0.7572 0.5414, -0.0388 -0.4829 -0.1502 -0.6641 -0.6174, -0.7016 0.5676 0.7943 -0.8770 -1.9943, 0.6359 1.0555 0.6047 0.8436 2.2208, 0.0236 0.4454 -0.2391 0.1376 -1.6154, -0.4678 -2.0165 0.6370 -0.2124 0.9803, 1.0565 0.1127 0.2131 -0.3955 -0.3299, 1.1327 0.9496 0.6333 1.1204 0.4342, -0.2545 0.2943 -2.2979 0.2329 -1.5774, -0.6871 -0....


In [84]:
X.size

res50: Int = 1000


In [86]:
X.map(_.split(" ")) //分出block，注意看数据类型

res52: Array[Array[String]] = Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.0420), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.1250, 2.3728, 0.2329, 0.1590, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.8240), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.0860, 0.6893, 0.0551, -1.2380), Array(-0.0299, 0.9467, -0.1940, 0.7572, 0.5414), Array(-0.0388, -0.4829, -0.1502, -0.6641, -0.6174), Array(-0.7016, 0.5676, 0.7943, -0.8770, -1.9943), Array(0.6359, 1.0555, 0.6047, 0.8436, 2.2208), Array(0.0236, 0.4454, -0....


In [87]:
X.map(_.split(" ")).map(_.map(_.toDouble))  //注意这是两层Array

res53: Array[Array[Double]] = Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.042), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.125, 2.3728, 0.2329, 0.159, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.824), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.086, 0.6893, 0.0551, -1.238), Array(-0.0299, 0.9467, -0.194, 0.7572, 0.5414), Array(-0.0388, -0.4829, -0.1502, -0.6641, -0.6174), Array(-0.7016, 0.5676, 0.7943, -0.877, -1.9943), Array(0.6359, 1.0555, 0.6047, 0.8436, 2.2208), Array(0.0236, 0.4454, -0.2391, 0....


In [88]:
X.map(_.split(" ").map(_.toDouble))  //这样是同样的效果，这是对里层的进行map

res54: Array[Array[Double]] = Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.042), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.125, 2.3728, 0.2329, 0.159, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.824), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.086, 0.6893, 0.0551, -1.238), Array(-0.0299, 0.9467, -0.194, 0.7572, 0.5414), Array(-0.0388, -0.4829, -0.1502, -0.6641, -0.6174), Array(-0.7016, 0.5676, 0.7943, -0.877, -1.9943), Array(0.6359, 1.0555, 0.6047, 0.8436, 2.2208), Array(0.0236, 0.4454, -0.2391, 0....


用.union将Array连成一个Array

注意：union不是可交换函数，不可在rdd的reduce里面用，但是可以在串型的reduce用

In [90]:
X.map(_.split(" ").map(_.toDouble)).reduce((x,y) => x.union(y))  //顺利连成了一个Array

res56: Array[Double] = Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473, 0.7119, -0.9113, 0.6411, 0.2659, 1.042, -2.1145, -0.3983, 0.5419, -0.0772, -0.8859, -0.4116, 1.0925, -1.5363, 0.5546, 1.7638, 0.4518, 1.6495, -0.9976, -0.0182, 1.1329, -0.125, 2.3728, 0.2329, 0.159, 0.9561, 0.1744, 0.9715, -0.2927, -1.0629, 1.4513, -1.1215, -0.7353, 1.0172, 0.6979, 0.824, -1.7683, 0.2096, 0.7109, -0.6189, 0.6227, 1.4214, 0.2154, 0.0395, 0.5862, -0.8251, -0.9125, 0.086, 0.6893, 0.0551, -1.238, -0.0299, 0.9467, -0.194, 0.7572, 0.5414, -0.0388, -0.4829, -0.1502, -0.6641, -0.6174, -0.7016, 0.5676, 0.7943, -0.877, -1.9943, 0.6359, 1.0555, 0.6047, 0.8436, 2.2208, 0.0236, 0.4454, -0.2391, 0.1376, -1.6154, -0.4678, -2.0165, 0.637, -0.2124, 0.9803, 1.0565, 0.1127, 0.2131, -0.3955, -0.3299, 1.1327, 0.9496, 0.6...


In [92]:
X.map(_.split(" ").map(_.toDouble)).foldLeft(Array(0.0)){(x,y) => x.union(y)}.drop(1)  //foldLeft需要给一个初值，记得把它扔掉（过河拆桥）

res58: Array[Double] = Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473, 0.7119, -0.9113, 0.6411, 0.2659, 1.042, -2.1145, -0.3983, 0.5419, -0.0772, -0.8859, -0.4116, 1.0925, -1.5363, 0.5546, 1.7638, 0.4518, 1.6495, -0.9976, -0.0182, 1.1329, -0.125, 2.3728, 0.2329, 0.159, 0.9561, 0.1744, 0.9715, -0.2927, -1.0629, 1.4513, -1.1215, -0.7353, 1.0172, 0.6979, 0.824, -1.7683, 0.2096, 0.7109, -0.6189, 0.6227, 1.4214, 0.2154, 0.0395, 0.5862, -0.8251, -0.9125, 0.086, 0.6893, 0.0551, -1.238, -0.0299, 0.9467, -0.194, 0.7572, 0.5414, -0.0388, -0.4829, -0.1502, -0.6641, -0.6174, -0.7016, 0.5676, 0.7943, -0.877, -1.9943, 0.6359, 1.0555, 0.6047, 0.8436, 2.2208, 0.0236, 0.4454, -0.2391, 0.1376, -1.6154, -0.4678, -2.0165, 0.637, -0.2124, 0.9803, 1.0565, 0.1127, 0.2131, -0.3955, -0.3299, 1.1327, 0.9496, 0.6...


探究drop函数：

.drop(n)

指删掉前几个数

In [93]:
Array(1,2,3,4,5,6).drop(2)

res59: Array[Int] = Array(3, 4, 5, 6)


In [94]:
Array(1,2,3,4,5,6).take(4)

res60: Array[Int] = Array(1, 2, 3, 4)


In [95]:
Array(1,2,3).union(Array(2,3,4))

res61: Array[Int] = Array(1, 2, 3, 2, 3, 4)


下面是写成map后的样子（和上面的结果一样）

In [63]:
import breeze.linalg._


val transLines0 = lines.map( s => {
    val rowsize = RowSize.value
    val columnsize = ColumnSize.value
    val columnlength = ColumnLength.value
    val p = columnsize * columnlength
    val Y = DenseVector(s.split("\n").map(_.split(",")(0).toDouble))
    val X = s.split("\n").map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)))
    (Y,X)
}
)

val transLines1 = lines.map( s => {
    val rowsize = RowSize.value
    val columnsize = ColumnSize.value
    val columnlength = ColumnLength.value
    val p = columnsize * columnlength
    val Y = DenseVector(s.split("\n").map(_.split(",")(0).toDouble))
    val X = s.split("\n").map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)).foldLeft(Array(0.0)){(x,y) => x.union(y)})
    (Y,X)
}
)


import breeze.linalg._
transLines0: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Array[Array[Double]]])] = MapPartitionsRDD[10] at map at <console>:40
transLines1: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Array[Double]])] = MapPartitionsRDD[11] at map at <console>:51


In [64]:
val transLines2 = lines.map( s => {
    val rowsize = RowSize.value
    val columnsize = ColumnSize.value
    val columnlength = ColumnLength.value
    val p = columnsize * columnlength
    val Y = DenseVector(s.split("\n").map(_.split(",")(0).toDouble))
    val X = s.split("\n").map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)).foldLeft(Array(0.0)){(x,y) => x.union(y)})
    (Y,X(0).drop(1))
}
)

transLines2: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Double])] = MapPartitionsRDD[12] at map at <console>:40


Mlib包生成随机数：

出来就是RDD

有：poissonRDD, logNormalRDD, exponentialRDD, normalRDD, uniformRDD, uniformVectorRDD

In [97]:
import org.apache.spark.SparkContext

import org.apache.spark.SparkContext


In [98]:
import org.apache.spark.mllib.random.RandomRDDs._

import org.apache.spark.mllib.random.RandomRDDs._


In [99]:
val u = poissonRDD(sc, 1, 1000000L, 10)  //100万个泊松分布的随机数，那个L要不要无所谓
//分布式计算的时候都要用到sc，这个对象是在初始化spark的时候会自动生成的(Spark context available as 'sc')

u: org.apache.spark.rdd.RDD[Double] = RandomRDD[15] at RDD at RandomRDD.scala:42


In [100]:
u.collect()

res62: Array[Double] = Array(0.0, 1.0, 1.0, 3.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 1.0, 2.0, 0.0, 0.0, 2.0, 2.0, 1.0, 0.0, 2.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 2.0, 1.0, 2.0, 2.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 3.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 1.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.0, 2.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 3.0, 1.0, 1.0, 2.0, 0.0, 0.0, 2.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 1.0, 3.0, 0.0, 1.0, 1.0, 1.0, 3.0...


In [101]:
u.collect().size

res63: Int = 1000000


In [34]:
import breeze.linalg._
val lines=sc.textFile("LinearModel_0.txt") 
val transLines0 = lines.map( s => {
    // val rowsize = RowSize.value
    // val columnsize = ColumnSize.value
    // val columnlength = ColumnLength.value
    // val p = columnsize * columnlength
    val Y = DenseVector(s.split("\n").map(_.split(",")(0).toDouble))  
    val X = s.split("\n").map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)))
    (Y,X)
}
)
val transLines1 = lines.map( s => {
    // val rowsize = RowSize.value
    // val columnsize = ColumnSize.value
    // val columnlength = ColumnLength.value
    // val p = columnsize * columnlength
    val Y = DenseVector(s.split("\n").map(_.split(",")(0).toDouble))
    val X = s.split("\n").map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)).foldLeft(Array(0.0)){(x,y) => x.union(y)})
    (Y,X)
}
)


import breeze.linalg._
lines: org.apache.spark.rdd.RDD[String] = LinearModel_0.txt MapPartitionsRDD[13] at textFile at <console>:29
transLines0: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Array[Array[Double]]])] = MapPartitionsRDD[14] at map at <console>:30
transLines1: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Array[Double]])] = MapPartitionsRDD[15] at map at <console>:40


In [3]:
transLines0.collect()

res0: Array[(breeze.linalg.DenseVector[Double], Array[Array[Array[Double]]])] = Array((DenseVector(3.0186),Array(Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.042), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.125, 2.3728, 0.2329, 0.159, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.824), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.086, 0.6893, 0.0551, -1.238), Array(-0.0299, 0.9467, -0.194, 0.7572, 0.5414), Array(-0.0388, -0.4829, -0.1502, -0.6641, -0.6174), Array(-0.7016, 0.5676, 0.7943, -0.877, -1.9943...


In [118]:
val lines=sc.textFile("LinearModel_0.txt") 
lines.collect().map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)))

lines: org.apache.spark.rdd.RDD[String] = LinearModel_0.txt MapPartitionsRDD[30] at textFile at <console>:31
res104: Array[Array[Array[Double]]] = Array(Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.042), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.125, 2.3728, 0.2329, 0.159, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.824), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.086, 0.6893, 0.0551, -1.238), Array(-0.0299, 0.9467, -0.194, 0.7572, 0.5414), Array(-0.0388, -0.4829, -0.1502, -0.6641, -0.6174), Array...


In [18]:
lines.collect()(1)(37550)

res14: Char = 6


In [33]:
lines.collect().map(_.split("\n")).take(1)

res29: Array[Array[String]] = Array(Array(3.0186,-1.2721 0.3226 1.4547 -0.6254 0.1473,0.7119 -0.9113 0.6411 0.2659 1.0420,-2.1145 -0.3983 0.5419 -0.0772 -0.8859,-0.4116 1.0925 -1.5363 0.5546 1.7638,0.4518 1.6495 -0.9976 -0.0182 1.1329,-0.1250 2.3728 0.2329 0.1590 0.9561,0.1744 0.9715 -0.2927 -1.0629 1.4513,-1.1215 -0.7353 1.0172 0.6979 0.8240,-1.7683 0.2096 0.7109 -0.6189 0.6227,1.4214 0.2154 0.0395 0.5862 -0.8251,-0.9125 0.0860 0.6893 0.0551 -1.2380,-0.0299 0.9467 -0.1940 0.7572 0.5414,-0.0388 -0.4829 -0.1502 -0.6641 -0.6174,-0.7016 0.5676 0.7943 -0.8770 -1.9943,0.6359 1.0555 0.6047 0.8436 2.2208,0.0236 0.4454 -0.2391 0.1376 -1.6154,-0.4678 -2.0165 0.6370 -0.2124 0.9803,1.0565 0.1127 0.2131 -0.3955 -0.3299,1.1327 0.9496 0.6333 1.1204 0.4342,-0.2545 0.2943 -2.2979 0.2329 -1.5774,-0.6871...


In [116]:
Array("2342,324,","342,432").map(_.split("\n"))

res102: Array[Array[String]] = Array(Array(2342,324,), Array(342,432))


In [105]:
transLines0.collect().take(1)(0)._2.size

res93: Int = 1


In [85]:
lines.collect().map(_.split("\n"))

res75: Array[Array[String]] = Array(Array(3.0186,-1.2721 0.3226 1.4547 -0.6254 0.1473,0.7119 -0.9113 0.6411 0.2659 1.0420,-2.1145 -0.3983 0.5419 -0.0772 -0.8859,-0.4116 1.0925 -1.5363 0.5546 1.7638,0.4518 1.6495 -0.9976 -0.0182 1.1329,-0.1250 2.3728 0.2329 0.1590 0.9561,0.1744 0.9715 -0.2927 -1.0629 1.4513,-1.1215 -0.7353 1.0172 0.6979 0.8240,-1.7683 0.2096 0.7109 -0.6189 0.6227,1.4214 0.2154 0.0395 0.5862 -0.8251,-0.9125 0.0860 0.6893 0.0551 -1.2380,-0.0299 0.9467 -0.1940 0.7572 0.5414,-0.0388 -0.4829 -0.1502 -0.6641 -0.6174,-0.7016 0.5676 0.7943 -0.8770 -1.9943,0.6359 1.0555 0.6047 0.8436 2.2208,0.0236 0.4454 -0.2391 0.1376 -1.6154,-0.4678 -2.0165 0.6370 -0.2124 0.9803,1.0565 0.1127 0.2131 -0.3955 -0.3299,1.1327 0.9496 0.6333 1.1204 0.4342,-0.2545 0.2943 -2.2979 0.2329 -1.5774,-0.6871...


In [93]:
"edfs".split("\f")

res83: Array[String] = Array(edfs)


In [108]:
lines.collect().map(_.split(",")(0).toDouble)

res95: Array[Double] = Array(3.0186, 7.3388, 10.8161, 8.3181, 1.8949, -3.1727, 2.6704, 4.8184, 2.5007, 0.3244, 7.0447, -1.071, -7.0225, 11.0142, -4.9931, 2.0444, 9.385, 1.4984, -3.9281, 8.3272, 4.3265, -0.8351, -4.9836, 0.8607, -8.2197, -0.6733, -5.4849, -2.3658, -8.1595, -0.8328, 8.3586, 6.4878, -10.67, -7.3739, 9.4501, -9.1718, 8.6345, -2.459, 5.2829, -7.5602, -1.9033, 2.8551, -8.1896, 3.3794, -1.2252, -4.5048, 14.6333, 7.5376, 5.4843, 5.093, -0.6671, 3.3756, 8.6578, 3.2509, 1.6056, -11.7849, 0.0032, -3.4953, 0.0175, -1.6445, 10.6872, -4.8624, 2.8222, 3.7436, -1.3321, -6.8358, -4.7346, 0.5778, -0.2466, 3.8284, 2.1126, -4.2085, -10.5775, 0.1889, -6.5307, 9.8019, -10.9353, 2.8733, 1.9336, 13.5181, 13.533, -12.3821, 1.8734, -9.4181, 7.676, 2.3874, -3.8178, 4.6795, 2.9945, 2.6317, -9.3035...


In [98]:
import breeze.linalg._
val lines=sc.textFile("LinearModel_0.txt") 
val transLines0 = lines.map( s => {
    // val rowsize = RowSize.value
    // val columnsize = ColumnSize.value
    // val columnlength = ColumnLength.value
    // val p = columnsize * columnlength
    val Y = DenseVector(s.split("\n").map(_.split(",")(0).toDouble))  
    val X = s.split("\n").map(_.split(",").drop(1).map(_.split(" ").map(_.toDouble)))
    (Y,X)
}
)
transLines0.collect()

transLines0: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Array[Array[Double]]])] = MapPartitionsRDD[22] at map at <console>:31
res86: Array[(breeze.linalg.DenseVector[Double], Array[Array[Array[Double]]])] = Array((DenseVector(3.0186),Array(Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.042), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.125, 2.3728, 0.2329, 0.159, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.824), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.086, 0.6893, 0.0551, -1....


In [97]:
val transLines0 = lines.map( s => {
    // val rowsize = RowSize.value
    // val columnsize = ColumnSize.value
    // val columnlength = ColumnLength.value
    // val p = columnsize * columnlength
    val Y = DenseVector(s.split(",")(0).toDouble) 
    val X = s.split(",").drop(1).map(_.split(" ").map(_.toDouble))
    (Y,X)
    
}
)
transLines0.collect()

transLines0: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Array[Double]])] = MapPartitionsRDD[21] at map at <console>:31
res85: Array[(breeze.linalg.DenseVector[Double], Array[Array[Double]])] = Array((DenseVector(3.0186),Array(Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473), Array(0.7119, -0.9113, 0.6411, 0.2659, 1.042), Array(-2.1145, -0.3983, 0.5419, -0.0772, -0.8859), Array(-0.4116, 1.0925, -1.5363, 0.5546, 1.7638), Array(0.4518, 1.6495, -0.9976, -0.0182, 1.1329), Array(-0.125, 2.3728, 0.2329, 0.159, 0.9561), Array(0.1744, 0.9715, -0.2927, -1.0629, 1.4513), Array(-1.1215, -0.7353, 1.0172, 0.6979, 0.824), Array(-1.7683, 0.2096, 0.7109, -0.6189, 0.6227), Array(1.4214, 0.2154, 0.0395, 0.5862, -0.8251), Array(-0.9125, 0.086, 0.6893, 0.0551, -1.238), Array(-0.0299,...


In [127]:
val transLines1 = lines.map( s => {
    val Y = DenseVector(s.split(",")(0).toDouble)
    val X = s.split(",").drop(1).map(_.split(" ").map(_.toDouble)).foldLeft(Array(0.0)){(x,y) => x.union(y)}
    (Y,X.drop(1))
}
)
transLines1.collect()

transLines1: org.apache.spark.rdd.RDD[(breeze.linalg.DenseVector[Double], Array[Double])] = MapPartitionsRDD[37] at map at <console>:31
res112: Array[(breeze.linalg.DenseVector[Double], Array[Double])] = Array((DenseVector(3.0186),Array(-1.2721, 0.3226, 1.4547, -0.6254, 0.1473, 0.7119, -0.9113, 0.6411, 0.2659, 1.042, -2.1145, -0.3983, 0.5419, -0.0772, -0.8859, -0.4116, 1.0925, -1.5363, 0.5546, 1.7638, 0.4518, 1.6495, -0.9976, -0.0182, 1.1329, -0.125, 2.3728, 0.2329, 0.159, 0.9561, 0.1744, 0.9715, -0.2927, -1.0629, 1.4513, -1.1215, -0.7353, 1.0172, 0.6979, 0.824, -1.7683, 0.2096, 0.7109, -0.6189, 0.6227, 1.4214, 0.2154, 0.0395, 0.5862, -0.8251, -0.9125, 0.086, 0.6893, 0.0551, -1.238, -0.0299, 0.9467, -0.194, 0.7572, 0.5414, -0.0388, -0.4829, -0.1502, -0.6641, -0.6174, -0.7016, 0.5676, 0...


In [122]:
Array(Array(1.0,1.0),Array(2.0,2.0)).foldLeft(Array(0.0)){_.union(_)}

res107: Array[Double] = Array(0.0, 1.0, 1.0, 2.0, 2.0)
